In [1]:
import os

current_directory = os.getcwd()
print("Current working directory:", current_directory)

import sympy
from sympy.utilities.lambdify import lambdify
from sympy import MutableDenseMatrix
from sympy import symbols, Matrix, latex, Add
from sympy.vector import *
from sympy import Matrix, symbols, pprint, simplify, atan2, sqrt,  eye, symbols
# from sympy import diag, shape, BlockMatrix, transpose, zeros
from sympy.utilities.codegen import codegen
import numpy as np
import inspect
import os
import re
import sys
from itertools import chain
from termcolor import colored
from IPython.display import display
sympy.init_printing()

WANNA_CREATE= True # False # True
WANNA_PRINT = False
max_iteration = 1 # to simplify expression

Current working directory: /home/cp/Dropbox/Ubuntu/ModelBasedCode/fishing_rod/LumpedParam


In [2]:
def get_function_args_np(func, name_fun, folder_name, *args):
    """_summary_

    Args:
        func (sympy): _description_
        name_fun (string): _description_
        params (sympy, optional): _description_. Defaults to None.
    """

    func_str = lambdify(args, func, modules=['numpy'])
    func_str.__name__ = name_fun
    func_str = inspect.getsource(func_str)
    func_str = func_str.replace('sin', 'np.sin').replace('cos', 'np.cos').\
                replace('acos', 'np.arccos').replace('sqrt', 'np.sqrt').replace('abs', 'np.abs').\
                replace('array', 'np.array').replace("arctan2","np.arctan2").replace("pi", "np.pi").replace("sign", "np.sign").\
                replace('arcnp.cos' ,'np.arccos')
    func_str = re.sub(r'def _lambdifygenerated', 'def ' + name_fun, func_str)

    if WANNA_CREATE:
        if os.path.isfile(folder_name + '/' + name_fun + '_np' + '.py'):
            print(colored(name_fun + '.py already exists!', 'yellow'))
            print(colored('===================================================================================================', 'yellow'))
        else:
            with open(folder_name + '/' + name_fun + '_np' + '.py', 'w') as f:
                f.write('import numpy as np\n')
                f.write(func_str)
            print(colored(name_fun + '_np' + '.py' + ' MATRIX ... DONE!', 'green'))
            print(colored('===================================================================================================', 'green'))

def get_function_args_fast(func, name_fun, folder_name, *args):
    """_summary_

    Args:
        func (sympy): _description_
        name_fun (string): _description_
        params (sympy, optional): _description_. Defaults to None.
    """

    func_str = lambdify(args, func, modules=['numpy'])
    func_str.__name__ = name_fun
    func_str = inspect.getsource(func_str)
    func_str = func_str.replace('sin', 'jnp.sin').replace('arccos', 'jnp.arccos').replace('cos', 'jnp.cos'). \
        replace('arcjnp.cos', 'jnp.arccos').replace('jnp.jnp', 'jnp').replace('sqrt', 'jnp.sqrt').replace('abs', 'jnp.abs'). \
        replace('array', 'np.array').replace("arctan2", "jnp.arctan2").replace("pi", "jnp.pi").replace("sign",
                                                                                                      "jnp.sign")
    func_str = re.sub(r'def _lambdifygenerated', 'def ' + name_fun, func_str)

    if WANNA_CREATE:
        if os.path.isfile(folder_name + '/' + name_fun + '.py'):
            print(colored(name_fun + '.py already exists!', 'yellow'))
            print(colored('===================================================================================================',
                          'yellow'))
        else:
            file_content = []
            file_content.append('import numpy as np\n')
            file_content.append('from jax import jit\n')
            file_content.append('from jax import numpy as jnp\n\n')
            file_content.append(func_str)

            with open(folder_name + '/' + name_fun + '.py', 'w') as f:
                f.writelines(file_content)

            print(colored(name_fun + '.py' + ' MATRIX ... DONE!', 'green'))
            print(colored('===================================================================================================',
                          'green'))


def get_function_args(func, name_fun, folder_name, *args):
    """_summary_

    Args:
        func (sympy): _description_
        name_fun (string): _description_
        params (sympy, optional): _description_. Defaults to None.
    """

    func_str = lambdify(args, func, modules=['numpy'])
    func_str.__name__ = name_fun
    func_str = inspect.getsource(func_str)
    func_str = func_str.replace('sin', 'jnp.sin').replace('arccos','jnp.arccos').replace('cos', 'jnp.cos').\
                replace('arcjnp.cos', 'jnp.arccos').replace('jnp.jnp', 'jnp').replace('sqrt', 'jnp.sqrt').replace('abs', 'jnp.abs').\
                replace('array', 'np.array').replace("arctan2","jnp.arctan2").replace("pi", "jnp.pi").replace("sign", "jnp.sign")
    func_str = re.sub(r'def _lambdifygenerated', 'def ' + name_fun, func_str)

    if WANNA_CREATE:
        if os.path.isfile(folder_name + '/' + name_fun + '.py'):
            print(colored(name_fun + '.py already exists!', 'yellow'))
            print(colored('===================================================================================================', 'yellow'))
        else:
            with open(folder_name + '/' + name_fun + '.py', 'w') as f:
                f.write('import numpy as np\n')
                f.write('from jax import jit\n')
                f.write('from jax import numpy as jnp\n\n')
                f.write(func_str)
            print(colored(name_fun + '.py' + ' MATRIX ... DONE!', 'green'))
            print(colored('===================================================================================================', 'green'))


def christoffel(M, q, q_dot):
    n = len(q)
    c = np.zeros((n, n, n), dtype=object)
    C = sympy.zeros(n, n)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                c[i,j,k] = (sympy.Rational(1, 2) * (sympy.diff(M[i,k], q[j]) + sympy.diff(M[i,j], q[k]) - sympy.diff(M[j,k], q[i])))
                C[i,j] += c[i,j,k] * q_dot[k]
    return C

def henkel_matrix(n, symbol=True):
    if symbol:
        henkel = sympy.Matrix([[1/(i + j + 1) for j in range(n)] for i in range(n)])
        henkel[0,0] = 1
    else:
        henkel = [[1/(i + j + 1) for j in range(n)] for i in range(n)]
        henkel[0,0] = 1
    return henkel

In [3]:
N = 15 # 5 # 20
q = sympy.symbols('q_:{}'.format(N))
qDot = sympy.symbols('q_dot_:{}'.format(N))
qDDot = sympy.symbols('q_ddot_:{}'.format(N))
g0 = sympy.symbols('g0', real=True, positive=True)
grav = [0, -g0, 0]

m = sympy.Matrix(np.array(sympy.symbols('m_:{}'.format(N), real=True, positive=True)))
k = sympy.Matrix(np.array(sympy.symbols('k_:{}'.format(N), real=True, positive=True)))
d = sympy.Matrix(np.array(sympy.symbols('d_:{}'.format(N), real=True, positive=True)))
L = sympy.Matrix(np.array(sympy.symbols('L_:{}'.format(N), real=True, positive=True)))
I_zz = sympy.Matrix(np.array(sympy.symbols('I_zz_:{}'.format(N), real=True, positive=True)))
a = L/2

q = sympy.Matrix(np.array(q))
q_dot = sympy.Matrix(np.array(qDot))
q_ddot = sympy.Matrix(np.array(qDDot))

# print(q, '\n', np.shape(q), type(q))
print('===================================================================================================')
print('\tq_i in R^{}'.format(N))
print('\tThe number of Segments selected is i = {}'.format(N))
print('===================================================================================================')


	q_i in R^15
	The number of Segments selected is i = 15


In [4]:
print('\n')
print(colored('\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\', 'cyan'))
print(colored('                          INIT FLEXIBLE LINK -- LUMPED PARAMETERS -- DYNAMIC', 'cyan'))
print(colored('\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\', 'cyan'))
print('\n')

# print(colored('===================================================================================================','white'))
# N = input("Enter the number of segments integer: ")
# print(colored('===================================================================================================','white'))
# try:
#     N = int(N)
# except ValueError:
#     print(colored("Invalid input. Please enter an integer", 'red'))

folder_name = 'fishing_rod_{}_joints'.format(N)
folder_name_np = folder_name + '_np'
path = os.getcwd()

## Check if the folder exists
folder_path = os.path.join(path, folder_name)
if not os.path.exists(folder_path):
    ## Create the folder if it does not exist
    os.makedirs(folder_path)
    print(colored('===================================================================================================','green'))
    print(colored("Folder created successfully!",'green'))
    print(colored("Create:\n{}".format(folder_path),'green'))
    print(colored('===================================================================================================', 'green'))
else:
    # os.makedirs(folder_path + str(int(np.floor(np.random.rand(1) * 100))))
    print(colored('===================================================================================================','yellow'))
    print(colored("Folder already exists!", 'yellow'))
    # print(colored("Create:\n{}".format(folder_path),'yellow'))
    print(colored('===================================================================================================','yellow'))

folder_path = os.path.join(path, folder_name_np)
if not os.path.exists(folder_name_np):
    ## Create the folder if it does not exist
    os.makedirs(folder_name_np)
    print(colored('===================================================================================================','green'))
    print(colored("Folder created successfully!",'green'))
    print(colored("Create:\n{}".format(folder_name_np),'green'))
    print(colored('===================================================================================================', 'green'))
else:
    # os.makedirs(folder_path + str(int(np.floor(np.random.rand(1) * 100))))
    print(colored('===================================================================================================','yellow'))
    print(colored("Folder already exists!", 'yellow'))
    # print(colored("Create:\n{}".format(folder_path),'yellow'))
    print(colored('===================================================================================================','yellow'))
    



\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
                          INIT FLEXIBLE LINK -- EULER BERNOULLI -- DYNAMIC
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\


Folder already exists!
Folder already exists!


In [5]:
mass_diag = sympy.diag([m])
K = sympy.diag([k])
D = sympy.diag([d])
inertia_matrix = sympy.diag([I_zz])

inAtt = sympy.Float(1)
A_act = sympy.zeros(N + 1)
A_act[0] = inAtt

henkel = henkel_matrix(N)

In [6]:
p_all = [] 
v_all = []
omega_all = []
p_old = sympy.Matrix(np.array([0,0,0]))
# assuming all the mass in the motor at the base 
for i in range(N):
    p_i = p_old + sympy.Matrix(np.array([L[i] * sympy.cos(sum(q[0:i+1])), L[i] * sympy.sin(sum(q[0:i+1])), 0]))
    p_all.append(p_i)
    p_old = p_i
    J_i = p_i.jacobian(q)
    v_i = sympy.Matrix(J_i) * sympy.Matrix(q_dot)
    v_all.append(v_i)
    omega_i = sympy.Matrix(np.array([0, 0, sum(q[:i+1])]))
    omega_all.append(omega_i)

v_all = sympy.Matrix(np.squeeze(v_all))
p_all = sympy.Matrix(np.squeeze(p_all))

p_tip = p_i
v_tip = v_i
omega_tip = omega_i


In [ ]:
J_p = p_tip.jacobian(q)
J_o = omega_tip.jacobian(q)
J = J_p.col_join(J_o)
# J = simplify(J, rational=True, max_iter=max_iteration)

# get_function_args(p_tip, '_p_tip', folder_name, q, L)
# get_function_args_np(p_tip, '_p_tip', folder_name_np, q, L)
# get_function_args(J, '_jacobian_diff', folder_name, q, L)
# get_function_args_np(J, '_jacobian_diff', folder_name_np, q, L)


In [ ]:
E_kin = 0
U_mgh = 0

for i in range(0, N):
    app_e_kin = sympy.Rational(1, 2) * m[i] * v_all[i,:] * sympy.transpose(v_all[i,:]) 
    app_e_kin_rot = sympy.Rational(1, 2) * I_zz[i] * q_dot[i]**2
    E_kin += app_e_kin[0, 0] + app_e_kin_rot
    app_mg = sympy.Matrix(sympy.Rational(1, 2) * m[i] * sympy.transpose(sympy.Matrix([0,-g0, 0])) * sympy.transpose(p_all[i,:]))
    U_mgh += app_mg[0,0]
Lag = E_kin + U_mgh

In [ ]:
B = sympy.Matrix(sympy.Matrix([sympy.diff(Lag, q_dot[i]) for i in range(len(q_dot))]).jacobian(q_dot))
# B = simplify(B, rational=True, max_iter=max_iteration)
G = sympy.Matrix([sympy.diff(U_mgh, q[i]) for i in range(len(q))])
C = christoffel(B, q, q_dot)

In [ ]:
# G_variables = G.free_symbols
# print('G depends on :\n\t', G_variables)
# p_tip_variables = p_tip.free_symbols
# print('p_tip depends on :\n\t', p_tip_variables)
# M_variables = B.free_symbols
# print('M depends on :\n\t', M_variables)
# K_variables = K.free_symbols
# print('K depends on :\n\t', K_variables)
# D_variables = D.free_symbols
# print('D depends on :\n\t', D_variables)

In [ ]:
# get_function_args_np(B, '_inertia_matrix', folder_name_np, q, L, m, I_zz)
# get_function_args_np(G, '_gravity_matrix', folder_name_np, q, L, m, I_zz, g0)
# get_function_args_np(D, '_damping_torque', folder_name_np, q_dot, d)
# get_function_args_np(K, '_stiffness_torque', folder_name_np, q, k)
# get_function_args(B, '_inertia_matrix', folder_name, q, L, m, I_zz)
# get_function_args(G, '_gravity_matrix', folder_name, q, L, m, I_zz, g0)
# get_function_args(D, '_damping_torque', folder_name, q_dot, d)
# get_function_args(K, '_stiffness_torque', folder_name, q, k)
get_function_args(C, '_coriolis_matrix', folder_name, q, q_dot, L, m, I_zz)
get_function_args_np(C, '_coriolis_matrix', folder_name_np, q, q_dot, L, m, I_zz)